In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import EIDA


# Hugging Face 패키지로 RoBERTa-base 모델, 토크나이저 로드
model_name = "roberta-base"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device) # layer 12개, d_model 768차원
# GLUE의 SST-2는 영화 리뷰에서 가져온 문장으로 감성 분석을 학습하는 데이터셋. label의 1은 긍정, 0은 부정
# 예시: {'sentence': "hide new secretions from the parental units", 'label': 0}
# label의 종류에 맞춰 num_labels=2 옵션으로 RoBERTa 모델을 생성. 그러면 RoBERTa의 사전학습 과정 masked language modeling에서 쓰이던 모델 마지막 부분의 pooler가 없어지고, 랜덤으로 초기화된 파라미터인 classifier가 생김. classifier는 두 계층(768에서 768으로, 768에서 2로)으로 구성되어 있음.


# Hugging Face 패키지로 GLUE 벤치마크의 SST-2 감정분석 데이터셋 로드
max_length = 72 # 모든 데이터를 손실 없이 다룰 수 있는 충분한 길이
batch_size = 16
dataset = load_dataset("glue", "sst2")
def preprocess_function(examples):
    return tokenizer(
        examples["sentence"],
        truncation=True,
        padding="max_length",
        max_length=max_length
    )
tokenized_dataset = dataset.map(preprocess_function, batched=True)

train_dataset = tokenized_dataset['train']
dev_dataset = tokenized_dataset['validation']


metric = evaluate.load("glue", "sst2") # GLUE 벤치마크에서 SST-2 셋에 대한 점수는 accuracy.
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# 모델의 원본 파라미터 중에서 classifier의 두 번째 계층(가중치 768x2, 편향 2) 부분을 제외한 모든 파라미터를 학습 불가능한 상태로 고정하고, 파라미터 옆에 추가할 어댑터 안에다가만 학습 가능한 파라미터를 둘 것임.
# 12개 레이어 각각의 W_Q, W_K, W_V, W_O, W_fc1, W_fc2들, 그리고 모델 마지막의 classifier의 첫번째 계층, 이렇게 73개의 파라미터는 뒤에서 EIDA.Linear_with_adapter 타입으로 교체할 때 requires_grad=False 설정이 이루어짐. 그러니 그 73개에 해당하지 않는 학습가능한 파라미터인 embedding과 layer normalization을 지금 고정함.
for name, param in model.roberta.embeddings.named_parameters(): # 모델의 embedding 부분
    param.requires_grad = False
for layer in model.roberta.encoder.layer: # 모델의 transformer layer 12개
    layer.attention.output.LayerNorm.weight.requires_grad = False
    layer.attention.output.LayerNorm.bias.requires_grad = False
    layer.output.LayerNorm.weight.requires_grad = False
    layer.output.LayerNorm.bias.requires_grad = False


for _ in range(4): # 4에폭의 학습을 수행하기 위한 반복문. 이 반복문은 다음과 같이 구성됨
    # 1. 73개의 파라미터 W 각각에 대해, W의 input token X와 Δoutput(학습과정에서 가중치 W의 gradient인 ΔW에 의한 output token Y의 변화량 ΔY=ΔW×Y를 의미)의 표본을 추출함. EIDA.forward_roberta 함수는 표본 토큰들을 sample_inputs, sample_delta_outputs 배열로 반환함.
    # 2. 추출된 표본을 가지고 PCA를 수행하여, SST-2 train set의 데이터를 RoBERTa-base에 통과시킬 때 각 latent space에서 나타나는 토큰의 분포를 저차원(32나 64) 부분공간으로 근사하여 표현함. EIDA.PCA 함수는 전체 latent space에서 추정된 저차원 부분공간으로 가는 projection map을 반환하도록 만들어져 있는데, 이것을 plane_inputs, plane_delta_outputs 배열에 저장함.
    # 3. 각 W에 대해, (W의 정의역에서 부분공간으로 가는 projection map), (X의 분포가 집중된 부분공간에서 ΔY의 분포가 집중된 부분공간으로 가는 학습 가능한 파라미터), (ΔY의 분포가 집중된 부분공간에서 공역으로 가는 projection map)^T 을 합성한 형태로 어댑터를 구성함. EIDA.Linear_with_adapter 타입이 이 형태를 지니도록 만들어짐.
    # 4. Hugging Face 패키지의 trainer를 이용해서 1에폭 학습을 수행함. 1에폭 단위로 linear schedule(처음에 warmup 구간, 이후에 decay 구간)을 따르게 됨.

    train_dataset = train_dataset.shuffle() # train set 섞기
    input_ids = torch.tensor(train_dataset['input_ids'])
    attention_mask = torch.tensor(train_dataset['attention_mask'])
    label = torch.tensor(train_dataset['label'])
    
    # token representation 표본 추출
    sample_inputs, sample_delta_outputs = EIDA.forward_roberta(model, input_ids, attention_mask, label, begin=0, end=256, batch_size=batch_size, max_length=max_length, N=2)
    # train set 67349개의 행 중에 end-begin=256개를 표본 추출에 활용함. vRAM 8GB짜리 RTX 4060로 모든 표본을 device='cuda'에 두고 작업 가능
    # 표본 추출을 수행하는 latent space는 49+73곳(input 49군데, output 73군데)의 위치 각각에서, 한 시퀀스(=문장, =데이터) 안에서 N=2개의 토큰을 표본으로 추출
    # sample_inputs: (end-start)*N = 512개의 벡터가 담긴 list 49개가 담긴 list
    # sample_delta_outputs: (end-start)*N = 512개의 벡터가 담긴 list 73개가 담긴 list

    plane_inputs=[]
    # 표본추출을 수행한 공간들(input token들이 머무르는 latent space 49개)을 가리키는 index:
    # 4*l+0: layer[l]에서 W_Q, W_K, W_V들의 공통 input의 위치 (768차원) (l = 0, 1, ..., 11)
    # 4*l+1: layer[l]에서 W_O의 input의 위치 (768차원)
    # 4*l+2: layer[l]에서 W_fc1의 input의 위치 (768차원)
    # 4*l+3: layer[l]에서 W_fc2의 input의 위치 (3072차원)
    # 4*12+0: classifier의 input의 위치 (768차원)
    list_input_64 = [0, 1, 5, 9, 13, 17, 21, 25, 29, 45] # 차원을 64로 둘 input 공간들의 index. 나머지는 32차원으로 둠.
    for i in range(4*12+1):
        plane_inputs.append(EIDA.PCA(sample_inputs[i], begin=0, end=len(sample_inputs[i]), plane_dim=64 if i in list_input_64 else 32, device=device)) # 주어진 512개의 토큰벡터로 주성분분석을 수행해서, 768(또는 3072)차원 공간에서 token representation의 분포를 가장 잘 포착하는 32(또는 64)차원 평면을 추정하는 함수
    del sample_inputs
    # plane_inputs는 [32(또는 64), 768(또는 3072)]의 shape을 가진 torch.tensor 49개의 list
    # 각 텐서는 latent space에서 PCA를 통해 추산된 32(또는 64)차원 부분공간으로의 projection map을 의미하고, 벡터 32(또는 64)개가 서로 orthonormal하게 있음. 이 맵이 어댑터의 A를 구성함.

    plane_delta_outputs=[]
    # 표본추출을 수행한 공간들(output token들이 머무르는 latent space 73개)을 가리기는 index:
    # 6*l+0: layer[l]에서 W_Q의 output의 위치 (768차원)
    # 6*l+1: layer[l]에서 W_K의 output의 위치 (768차원)
    # 6*l+2: layer[l]에서 W_V의 output의 위치 (768차원)
    # 6*l+3: layer[l]에서 W_O의 output의 위치 (768차원)
    # 6*l+4: layer[l]에서 W_fc1의 output의 위치 (3072차원)
    # 6*l+5: layer[l]에서 W_fc2의 output의 위치 (768차원)
    # 6*12+0: classifier의 첫번째 계층(가중치 768x768, 편향 768)의 output의 위치 (768차원)
    list_delta_output_64 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 46, 48, 49, 54, 55, 60, 66] # 차원을 64로 둘 output 공간들의 index. 나머지는 32차원으로 둠.
    for i in range(6*12+1):
        plane_delta_outputs.append(EIDA.PCA(sample_delta_outputs[i], begin=0, end=len(sample_delta_outputs[i]), plane_dim=64 if i in list_delta_output_64 else 32, device=device))
    del sample_delta_outputs
    # plane_delta_outputs는 [32(또는 64), 768(또는 3072)]의 shape을 가진 torch.tensor 73개의 list
    # 각 텐서는 latent space에서 PCA를 통해 추산된 32(또는 64)차원 부분공간으로의 projection map을 의미하고, 벡터 32(또는 64)개가 서로 orthonormal하게 있음. 이 맵의 transpose가 어댑터의 C를 구성함.


    # 모델 안의 73개의 파라미터를 각각 EIDA.Linear_with_adapter 타입으로 교체하는 과정
    # EIDA.Linear_with_adapter는 원본 파라미터의 weight을 (원본 파라미터의 weight) + (C @ B @ A) 로 바꿈(bias가 있다면 그대로 가져옴). 이 중에서 B만 학습가능한 파라미터로 둔다.
    # A는 파라미터의 input token이 머무는 latent space에서 32차원 부분공간으로의 projection map.
    # C는 파라미터의 output token이 머무는 latent space에서 64차원 부분공간으로의 projection map을 transpose해서 얻어진, 64차원에서 768(또는 3072)차원으로 가는 map.(각 열의 orthonormality 덕분에 transpose가 projection의 역과정이 됨)
    # B는 그 둘을 잇는, 32(또는 64)차원에서 32(또는 64)차원으로 가는 linear map. Δoutput의 분포보다는 input token의 분포가 훨씬 저차원의 구조를 잘 가지므로 input token의 분포는 대부분 32차원으로, Δoutput의 분포는 대부분 64차원으로 근사하였음.
    for l, layer in enumerate(model.roberta.encoder.layer):
        layer.attention.self.query = EIDA.Linear_with_adapter(original_param=layer.attention.self.query, A=plane_inputs[4*l+0], C=plane_delta_outputs[6*l+0])
        layer.attention.self.key = EIDA.Linear_with_adapter(original_param=layer.attention.self.key, A=plane_inputs[4*l+0], C=plane_delta_outputs[6*l+1])
        layer.attention.self.value = EIDA.Linear_with_adapter(original_param=layer.attention.self.value, A=plane_inputs[4*l+0], C=plane_delta_outputs[6*l+2])
        layer.attention.output.dense = EIDA.Linear_with_adapter(original_param=layer.attention.output.dense, A=plane_inputs[4*l+1], C=plane_delta_outputs[6*l+3])
        layer.intermediate.dense = EIDA.Linear_with_adapter(original_param=layer.intermediate.dense, A=plane_inputs[4*l+2], C=plane_delta_outputs[6*l+4])
        layer.output.dense = EIDA.Linear_with_adapter(original_param=layer.output.dense, A=plane_inputs[4*l+3], C=plane_delta_outputs[6*l+5])
    model.classifier.dense = EIDA.Linear_with_adapter(original_param=model.classifier.dense, A=plane_inputs[4*12+0], C=plane_delta_outputs[6*12+0])
    # 이제 학습 가능한 파라미터의 수: 73*(32*64)+(768*2+2) = 151,042개

    training_args = TrainingArguments(
        os.path.join("results", "roberta", "sst2"),
        save_strategy="no",
        learning_rate=2e-4,
        warmup_ratio=0.1,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=1,
        weight_decay=0.1,
        logging_dir="./logs",
        logging_steps=2100,
        eval_strategy="steps",
        eval_steps=2100,
        fp16=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train() # 학습 실행

    # 학습된 어댑터를 모델에 merge하고 모델을 저장하기
    for l, layer in enumerate(model.roberta.encoder.layer):
        layer.attention.self.query = layer.attention.self.query.merge()
        layer.attention.self.key = layer.attention.self.key.merge()
        layer.attention.self.value = layer.attention.self.value.merge()
        layer.attention.output.dense = layer.attention.output.dense.merge()
        layer.intermediate.dense = layer.intermediate.dense.merge()
        layer.output.dense = layer.output.dense.merge()
    model.classifier.dense = model.classifier.dense.merge()

    model.save_pretrained(os.path.join("results", "roberta", "sst2", "_"))